In [2]:
!pip install pygame numpy carla

In [1]:
!"N:\Programs\Git\bin\git.exe" config --global user.email "pranavmallela2020@gmail.com"
!"N:\Programs\Git\bin\git.exe" config --global user.name "Pranav Mallela"

In [1]:
import carla
import random
import time
import cv2
import numpy as np

# Initialize the frame counter globally
frame_counter = 0

def main():
    global frame_counter  # Ensure that we are modifying the global frame_counter

    # Connect to CARLA
    client = carla.Client('localhost', 2000)
    client.set_timeout(5.0)

    # Load a map
    world = client.get_world()

    # Enable synchronous mode
    settings = world.get_settings()
    settings.synchronous_mode = True
    settings.fixed_delta_seconds = 0.1  # Increased delta time
    world.apply_settings(settings)

    # Get blueprint library
    blueprint_library = world.get_blueprint_library()

    # Spawn the ego vehicle (Tesla Model 3)
    tesla_bp = blueprint_library.find('vehicle.tesla.model3')
    tesla_bp.set_attribute('color', '255,0,0')

    # Deterministic spawn point for ego vehicle
    spawn_points = world.get_map().get_spawn_points()
    ego_spawn_point = spawn_points[0]
    ego_vehicle = world.spawn_actor(tesla_bp, ego_spawn_point)

    # Set ego vehicle to autopilot
    ego_vehicle.set_autopilot(True)

    # Initialize BEV camera with reduced resolution
    camera_bp = blueprint_library.find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '640')
    camera_bp.set_attribute('image_size_y', '480')
    camera_bp.set_attribute('fov', '90')

    # Set camera transform (Bird's eye view)
    camera_transform = carla.Transform(carla.Location(z=15), carla.Rotation(pitch=-90))
    bev_camera = world.spawn_actor(camera_bp, camera_transform, attach_to=ego_vehicle)

    def process_image(image):
        global frame_counter  # Use the global frame_counter
        frame_counter += 1
        if frame_counter % 2 != 0:  # Display every other frame
            return

        # Convert image to array
        array = np.frombuffer(image.raw_data, dtype=np.uint8)
        array = array.reshape((image.height, image.width, 4))[:, :, :3]

        # Make the image writable for OpenCV operations
        array = np.ascontiguousarray(array)

        # Fetch speed and steering angle
        velocity = ego_vehicle.get_velocity()
        speed = (3.6 * (velocity.x**2 + velocity.y**2 + velocity.z**2) ** 0.5)  # Convert m/s to km/h
        control = ego_vehicle.get_control()
        steering_angle = control.steer

        # Display speed and steering angle on the image
        text_speed = f"Speed: {speed:.2f} km/h"
        text_steer = f"Steering: {steering_angle:.2f} rad"
        cv2.putText(array, text_speed, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(array, text_steer, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Resize the image to save memory (optional)
        array = cv2.resize(array, (400, 300))

        # Show the image
        cv2.imshow('BEV Camera View', array)

        # Close the window when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            raise StopIteration

    bev_camera.listen(process_image)

    # Spawn parked vehicles along the side of the road (more vehicles, and correct placement)
    parked_vehicles = []
    num_vehicles = 20  # Increased number of vehicles
    parking_spots = spawn_points[5:25:2]  # Using side-of-road spawn points

    for idx, point in enumerate(parking_spots[:num_vehicles]):
        if idx % 4 == 0:
            continue  # Leave every 4th spot open (for parking spots)
        vehicle_bp = random.choice(blueprint_library.filter('vehicle.*'))
        vehicle_bp.set_attribute('role_name', 'parked')
        point.location.y += 2.5  # Move vehicles to the side of the road
        vehicle = world.try_spawn_actor(vehicle_bp, point)
        if vehicle:
            parked_vehicles.append(vehicle)
            print(f"Parked vehicle spawned at: {point.location}")

    print(f"{num_vehicles} vehicles spawned. Press 'q' to stop.")

    try:
        while True:
            world.tick()
            time.sleep(0.1)  # Simulate a slower update rate to save memory
    except StopIteration:
        print("Stopping simulation...")
    finally:
        ego_vehicle.destroy()
        bev_camera.stop()
        bev_camera.destroy()
        for car in parked_vehicles:
            if car is not None:
                car.destroy()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


Parked vehicle spawned at: Location(x=-1.848311, y=-84.732613, z=0.275307)
Parked vehicle spawned at: Location(x=-117.493454, y=-0.721493, z=0.275307)
Parked vehicle spawned at: Location(x=-74.387177, y=28.234657, z=0.350000)
Parked vehicle spawned at: Location(x=-74.387177, y=61.931622, z=1.207297)
Parked vehicle spawned at: Location(x=149.592041, y=7.871150, z=0.275307)
Parked vehicle spawned at: Location(x=129.127945, y=7.871150, z=0.275307)
Parked vehicle spawned at: Location(x=183.285294, y=8.808762, z=0.275307)
20 vehicles spawned. Press 'q' to stop.


StopIteration: 

StopIteration: 

: 

: 